# Importing dependencies

In [ ]:
import pandas as pd
import numpy as np          
from sklearn.preprocessing import MinMaxScaler  
from sklearn.metrics import f1_score  
from sklearn.pipeline import make_pipeline  
from sklearn.impute import SimpleImputer  
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.compose import ColumnTransformer
from lightgbm import LGBMClassifier
from scipy.stats import hmean
from sklearn.multioutput import MultiOutputClassifier
import scipy
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.linear_model import RidgeClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import StratifiedGroupKFold
from pathlib import Path
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings("ignore")
from sklearn.experimental import enable_iterative_imputer  # This line enables it
from sklearn.impute import IterativeImputer

# Reading Data

In [6]:
"""

just drop the path of your new data directory nothing more than that. 


"""
path = r"C:\Users\Maab\Desktop\ADHD_Kaggle_Competition\Repo\WiDS-Datathon-2025\Data\raw"

def read_data(base_path:str) -> pd.DataFrame :
    path = Path(base_path)
    trc=pd.read_excel(path /'TRAIN_NEW'  / 'TRAIN_CATEGORICAL_METADATA_new.xlsx')
    trq=pd.read_excel(path /'TRAIN_NEW'  / 'TRAIN_QUANTITATIVE_METADATA_new.xlsx')
    trf=pd.read_csv(path   /'TRAIN_NEW'  / 'TRAIN_FUNCTIONAL_CONNECTOME_MATRICES_new_36P_Pearson.csv')
    trs=pd.read_excel(path /'TRAIN_NEW'  / 'TRAINING_SOLUTIONS.xlsx')  
    tsc=pd.read_excel(path /'TEST'      / 'TEST_CATEGORICAL.xlsx')
    tsq=pd.read_excel(path /'TEST'       / 'TEST_QUANTITATIVE_METADATA.xlsx')    
    tsf=pd.read_csv(path   /'TEST'       / 'TEST_FUNCTIONAL_CONNECTOME_MATRICES.csv')    
    sub=pd.read_excel(path / 'SAMPLE_SUBMISSION.xlsx')    
    dic=pd.read_excel(path /'Data Dictionary.xlsx')
    return trc, trq, trf, trs, tsc, tsq, tsf, sub, dic

trc, trq, trf, trs, tsc, tsq, tsf, sub, dic = read_data(base_path=path)

# Data Merging 
cq = pd.merge(trc, trq, on='participant_id', how='left')
feat = pd.merge(cq, trf, on='participant_id', how='left')  
qc = pd.merge(tsc, tsq, on='participant_id', how='left')
train = pd.merge(feat, trs, on='participant_id', how='left') 
test = pd.merge(qc, tsf, on='participant_id', how='left')

In [8]:
# highlighting important variables. 
#note that I did't deal with the quantative data trq as categorical
# I will use the OneHotEncoder for the categorical data as we have some data trap that I don't think we can use the label encoder for. 

for feature in trc.columns:
    train[feature] = train[feature].astype(object)
train_ids = train['participant_id']
test_ids = train['participant_id'] # I will store them for later usage in grouping in validation why?  I don't want the same user to appear in both train and test. 
num_feats = [feature for feature in train.columns if train[feature].dtype == 'float64']
cat_feats = [feature for feature in train.columns if train[feature].dtype == 'object'] # seperate categorical and numerical features help me reteriving them later easily for preprocessing.
target_cols = ['ADHD_Outcome', 'Sex_F']
groups = train_ids
log_features = [f for f in num_feats if (train[f] >= 0).all() and scipy.stats.skew(train[f]) > 0] # I will apply log transformation for the skewed features (basic preprocessing our models assume normal bell curve probability distribution)

encoder = OneHotEncoder(handle_unknown='ignore',sparse_output=False)
le = LabelEncoder()
for feature in log_features:
    train[feature] = np.log1p(train[feature])
    test[feature] = np.log1p(test[feature])
    
from sklearn.experimental import enable_iterative_imputer  # This line enables it
from sklearn.impute import IterativeImputer

# Define which features you want to impute (everything except target columns)
features_to_impute = [f for f in train.columns if f not in target_cols]

# Create the imputer
imputer = IterativeImputer(random_state=0)

# Fit the imputer on train set and transform both train and test
train[features_to_impute] = imputer.fit_transform(train[features_to_impute])
test[features_to_impute] = imputer.transform(test[features_to_impute])

# Convert all categorical features to strings to avoid mixed types
for feature in cat_feats:
    train[feature] = train[feature].astype(str)
    test[feature] = test[feature].astype(str)

for feature in cat_feats:
    if feature != 'participant_id':
        
        train_encoded = encoder.fit_transform(train[[feature]])
        test_encoded = encoder.transform(test[[feature]])
        print(f"Encoding {feature} with OneHotEncoder")
        train_encoded_df = pd.DataFrame(train_encoded, columns=encoder.get_feature_names_out([feature]))
        test_encoded_df = pd.DataFrame(test_encoded, columns=encoder.get_feature_names_out([feature]))
        train = pd.concat([train.drop(columns=[feature]), train_encoded_df], axis=1)
        test = pd.concat([test.drop(columns=[feature]), test_encoded_df], axis=1)

        
    else:
        train[feature] = le.fit_transform(train[feature])
        test[feature] = le.fit_transform(test[feature])        
  
for df in (train,test):
    df.drop(columns=['participant_id'], inplace=True)      

KeyError: 'participant_id'

# Preprocessing

In [118]:
#Scaling features
from sklearn.preprocessing import StandardScaler 
scaler = StandardScaler()

for col in train.columns:
    if col not in target_cols and col not in cat_feats:
        train[col] = scaler.fit_transform(train[[col]])
        test[col] = scaler.transform(test[[col]])
    

# Training and Validation

In [121]:
train[target_cols]

,ADHD_Outcome,Sex_F
0,1,0
1,1,0
2,0,1
3,0,1
4,1,0
...,...,...
1208,0,1
1209,1,0
1210,1,1
1211,1,0


In [ ]:
"""
ayo so there is this modeling thing, the part are 
- base model is a RidgeClassifier with a regulizer (more on that on comments)"
- multioutput classifier is just a wrapper it sucks the base model and elevate its skills to make it output multiple targets.
- there is a pipeline inside the multiooutput classifier that does the following:
1. impute missing values with mean
2. log transform the features that are skewed so bad (you know why now)
3. scale the features to be between 0 and 1
4. PCA to reduce the dimensionality of the features (970 components)
   
"""
features = test.columns
n_splits = 5 
cv = StratifiedGroupKFold(n_splits=n_splits)
##base_model = RidgeClassifier(random_state=7, class_weight="balanced")
base_model = LogisticRegression(random_state=7,penalty='l2', C=0.02,class_weight="balanced")

model = MultiOutputClassifier(base_model)

def validate(trainset, testset, target_cols):
    weights = (trainset['Sex_F'] == 1) & (trainset['ADHD_Outcome'] == 1) * 2 + 1
    model.fit(trainset.drop(columns=target_cols), trainset[target_cols],sample_weight=weights)
    pred = model.predict(testset.drop(columns=target_cols))
    valid_idx = testset[target_cols].notna().all(axis=1)
    
    valid_testset = testset.loc[valid_idx, target_cols]
    valid_pred = pred[valid_idx]
    f1_scores = [f1_score(valid_testset[col], valid_pred[:, i]) for i, col in enumerate(target_cols)]
    
    print(f"F1-scores per target: {dict(zip(target_cols, f1_scores))}")
    
    return f1_scores


stds = []
F1s = []

for train_index, test_index in cv.split(train.drop(columns=target_cols), train[target_cols[0]], groups=groups): 
    train_v, test_v = train.iloc[train_index], train.iloc[test_index]
    
    stds.append(test_v[target_cols].std())
    F1s.append(validate(train_v, test_v, target_cols))

F1s = np.array(F1s)

print("Mean F1-scores per target:", F1s.mean(axis=0))
print("F1-score stds per target:", F1s.std(axis=0))
print("Standard deviations of test sets:", stds)
print("score mean ", np.mean(F1s, axis=0))
score = hmean(F1s, axis=0)
score = hmean(score, axis=0)
print("overall harmonic mean ", score)

F1-scores per target: {'ADHD_Outcome': 0.7051671732522796, 'Sex_F': 0.5272727272727272}
F1-scores per target: {'ADHD_Outcome': 0.7283950617283951, 'Sex_F': 0.5}
F1-scores per target: {'ADHD_Outcome': 0.6729559748427673, 'Sex_F': 0.50920245398773}
F1-scores per target: {'ADHD_Outcome': 0.6729559748427673, 'Sex_F': 0.4875}
F1-scores per target: {'ADHD_Outcome': 0.7244582043343654, 'Sex_F': 0.5288753799392097}
Mean F1-scores per target: [0.70078648 0.51057011]
F1-score stds per target: [0.02404516 0.0158738 ]
Standard deviations of test sets: [ADHD_Outcome    0.466218
Sex_F           0.480409
dtype: float64, ADHD_Outcome    0.464573
Sex_F           0.472377
dtype: float64, ADHD_Outcome    0.466218
Sex_F           0.475213
dtype: float64, ADHD_Outcome    0.465098
Sex_F           0.468331
dtype: float64, ADHD_Outcome    0.465098
Sex_F           0.480850
dtype: float64]
score mean  [0.70078648 0.51057011]
overall harmonic mean  0.5901173137436236


# Inference and Submission

In [123]:
def inference(testdata, model):

    model.fit(train.drop(columns=["ADHD_Outcome", "Sex_F"], axis=1), train[["ADHD_Outcome", "Sex_F"]])
    y_pred = model.predict(test)
    sub['ADHD_Outcome'] = y_pred[:, 0] 
    sub['Sex_F'] = y_pred[:, 1]        
    sub.to_csv(f'../submissions/submission{score}.csv', index=False)
    
inference(test, model)